In [1]:
'''
首先，我们用[pandas]读取数据，就像我们在第1部分中所做的那样。
与第1部分不同，我们现在使用unlabeledTrain.tsv，其中包含50,000个没有标签的额外评论。
由于Word2Vec可以从未标记的数据中学习，现在可以使用这些额外的50,000条评论。
'''
import pandas as pd

# Read data from files 
train = pd.read_csv( "data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews and %d unlabeled reviews\n" % (train["review"].size, \
                test["review"].size, unlabeled_train["review"].size) )

'''
我们编写的用于清理数据的函数也类似于第1部分，尽管现在有一些差异。
首先，为了训练Word2Vec，最好不要去掉停用词，因为算法依赖于更广泛的句子上下文来产生高质量的词向量
'''

# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    # 将文档转换为一系列单词的功能，可选地删除停用词。 返回单词列表。
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

'''
接下来，我们需要一个特定的输入格式。
Word2Vec期望单个句子，每个单词作为单词列表。 换句话说，输入格式是一个列表的列表(a list of list)。
如何将一个段落分成句子并不是一帆风顺的。
自然语言中有各种各样的陷阱。 英语句子可以 以 “？”，“！”，“”“或 ”。“结尾，间距和大写也不是可靠的指南。
因此，我们将使用NLTK的punkt标记器进行句子分割 。为了使用这个，你需要安装NLTK并使用 nltk.download（）来下载punkt的相关训练文件。
'''
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

# apply this function to prepare our data for input to Word2Vec
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
print("finished")
print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
print("finished")

'''
这里要注意 += 和 append
需要注意的细节是Python列表中“+ =”和“append”的区别。 在许多应用中，两者是可以互换的，但是在这里它们不是。 
如果您将列表追加到列表的另一个列表中，“append”将只追加第一个列表; 你需要使用“+ =”来一次加入所有的列表。
'''

# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality
min_word_count = 40   # Minimum word count
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

print("finished")

Read 25000 labeled train reviews, 25000 labeled test reviews and 50000 unlabeled reviews

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
Parsing sentences from training set


/home/hao/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/hao/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/home/hao/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/home/hao/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beau

finished
Parsing sentences from unlabeled set


/home/hao/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/hao/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/hao/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like request

finished


...

Training model...


2017-12-05 16:43:46,157 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2017-12-05 16:43:46,218 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2017-12-05 16:43:46,290 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2017-12-05 16:43:46,361 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2017-12-05 16:43:46,420 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2017-12-05 16:43:46,487 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2017-12-05 16:43:46,557 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2017-12-05 16:43:46,615 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2017-12-05 16:43:46,674 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

2017-12-05 16:43:50,629 : INFO : PROGRESS: at sentence #760000, processed 16990622 words, keeping 120930 word types
2017-12-05 16:43:50,703 : INFO : PROGRESS: at sentence #770000, processed 17217759 words, keeping 121703 word types
2017-12-05 16:43:50,779 : INFO : PROGRESS: at sentence #780000, processed 17447905 words, keeping 122402 word types
2017-12-05 16:43:50,848 : INFO : PROGRESS: at sentence #790000, processed 17674981 words, keeping 123066 word types
2017-12-05 16:43:50,880 : INFO : collected 123504 word types from a corpus of 17798082 raw words and 795538 sentences
2017-12-05 16:43:50,880 : INFO : Loading a fresh vocabulary
2017-12-05 16:43:50,954 : INFO : min_count=40 retains 16490 unique words (13% of original 123504, drops 107014)
2017-12-05 16:43:50,955 : INFO : min_count=40 leaves 17238940 word corpus (96% of original 17798082, drops 559142)
2017-12-05 16:43:51,037 : INFO : deleting the raw counts dictionary of 123504 items
2017-12-05 16:43:51,041 : INFO : sample=0.001 d

2017-12-05 16:44:56,789 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-12-05 16:44:56,790 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-12-05 16:44:56,795 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-12-05 16:44:56,796 : INFO : training on 88990410 raw words (63750505 effective words) took 65.4s, 974348 effective words/s
2017-12-05 16:44:56,796 : INFO : precomputing L2-norms of word weight vectors
2017-12-05 16:44:56,942 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2017-12-05 16:44:56,943 : INFO : not storing attribute syn0norm
2017-12-05 16:44:56,944 : INFO : not storing attribute cum_table
2017-12-05 16:44:59,360 : INFO : saved 300features_40minwords_10context


finished


In [5]:
# Vector Averaging

import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [6]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

/home/hao/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/hao/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [8]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "data/Word2Vec_AverageVectors_to_RForest.csv", index=False, quoting=3 )
print("Finished")

Fitting a random forest to labeled training data...
Finished


In [10]:
trainDataVecs.shape

(25000, 300)

In [11]:
testDataVecs.shape

(25000, 300)

In [ ]:
'''
下面是使用KNN聚类之后的随机森林
'''

In [22]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] // 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  450.346568107605 seconds.


In [32]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
            
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1
    
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "result/BagOfCentroids.csv", index=False, quoting=3 )


Cluster 0

Cluster 1

Cluster 2

Cluster 3

Cluster 4

Cluster 5

Cluster 6

Cluster 7

Cluster 8

Cluster 9
Fitting a random forest to labeled training data...
